# IRP Integration Demo

This notebook showcases usage of the `irp_integration` module. 

### Imports & Setup

In [ ]:
%load_ext autoreload
%autoreload 2
from system.irp_integration import IRPClient
import json, os

irp_client = IRPClient()

### Configurations

In [18]:
edm_name = "integration_demo" # ensure unique within IRP
portfolio_name = "demo_portfolio" # ensure unique within the edm
accounts_file_name = "accounts.csv"
locations_file_name = "locations.csv"
analysis_job_name = "basic_job"
analysis_profile_name = "North Atlantic Hurricane 18.0 EP Distributed Wind Only (Tutorial)"
output_profile_name = "Default"
event_rate_scheme_name = "RMS 2023 Stochastic Event Rates"

### Create EDM

In [3]:
print(json.dumps(irp_client.edm.create_edm(edm_name, "databridge-1"), indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751564
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751564
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751564
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751564
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751564
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751564
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751564
Workflow status: RUNNING; Percent complete: 30
Polling workflow url https://api-euw1.rms-ppe.com/risk

### Create Portfolio

In [4]:
portfolio_response = irp_client.portfolio.create_portfolio(edm_name, portfolio_name)
portfolio_id = portfolio_response['id'].split('/')[-1]
print(json.dumps(portfolio_response, indent=2))

{
  "id": "https://api-euw1.rms-ppe.com/riskmodeler/v1/portfolios/1"
}


### MRI Import

In [10]:
current_dir = os.path.dirname(os.path.abspath(''))
working_files_dir = os.path.join(current_dir, "_Tools", "files", "working_files")

bucket_response = irp_client.mri_import.create_aws_bucket()
bucket_url = bucket_response.headers['location']
bucket_id = bucket_url.split('/')[-1]

accounts_file_path = os.path.join(working_files_dir, accounts_file_name)
accounts_credentials = irp_client.mri_import.get_file_credentials(bucket_url, accounts_file_name, 100, "account")
irp_client.mri_import.upload_file_to_s3(accounts_credentials, accounts_file_path)

locations_file_path = os.path.join(working_files_dir, locations_file_name)
locations_credentials = irp_client.mri_import.get_file_credentials(bucket_url, locations_file_name, 100, "location")
irp_client.mri_import.upload_file_to_s3(locations_credentials, locations_file_path)

mapping_file_name = "mapping.json"
mapping_file_path = os.path.join(working_files_dir, mapping_file_name)
mapping_file_id = irp_client.mri_import.upload_mapping_file(mapping_file_path, bucket_id).json()

import_response = irp_client.mri_import.execute_mri_import(
    edm_name,
    int(portfolio_id),
    int(bucket_id),
    accounts_credentials['file_id'],
    locations_credentials['file_id'],
    mapping_file_id
)
print(json.dumps(import_response, indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751593
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751593
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751593
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751593
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751593
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751593
Workflow status: RUNNING; Percent complete: 50
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751593
Workflow status: RUNNING; Percent complete: 50
Polling workflow url https://api-euw1.rms-ppe.com/riskmode

### Geocode

In [11]:
print(json.dumps(irp_client.portfolio.geocode_portfolio(edm_name, portfolio_id), indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751597
Workflow status: RUNNING; Percent complete: 10
Polling workflow url https://api-euw1.rms-ppe.com/risk

### Analysis

In [19]:
analysis_response = irp_client.analysis.execute_analysis(
    analysis_job_name,
    edm_name,
    portfolio_id,
    analysis_profile_name,
    output_profile_name,
    event_rate_scheme_name
)
print(json.dumps(analysis_response, indent=2))

Submitting workflow request...
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: QUEUED; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodeler/v1/workflows/22751616
Workflow status: RUNNING; Percent complete: 0
Polling workflow url https://api-euw1.rms-ppe.com/riskmodele